In [1]:
!pip install monai -q --no-index --find-links=../input/vesuvis-downloads
!python -m pip install -q /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl --no-index --find-links=/kaggle/input/omegaconf222py3/

In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from tqdm.contrib.concurrent import thread_map
from tqdm.asyncio import tqdm as async_tqdm
from monai.inferers import SlidingWindowInferer
import pytorch_lightning as pl

import torch
import os
from typing import Union, Dict, Tuple
import cv2

2024-04-28 16:10:47.710411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 16:10:47.710518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 16:10:47.852044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Check if TPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('Running on GPU')
else:
    device = torch.device("cpu")
    print('Running on CPU')

# Print the number of replicas
print("Number of replicas: ", torch.cuda.device_count() if torch.cuda.is_available() else 1)

Running on GPU
Number of replicas:  1


## **DataSet Preparation**

In [6]:
MAIN_PATH = '/kaggle/input/blood-vessel-segmentation'
TEST_PATH = MAIN_PATH + '/test'
VALIDATION = MAIN_PATH + '/train/kidney_1_dense'
WEIGHTS_PATH = '/kaggle/input/unet-bdou-loss-weights/UNet_BoundaryDoULoss_size_1_512_bs32_hard/0/checkpoints/last.ckpt'
INPUT_CHANNELS, OUTPUT_CHANNELS = 1,1

In [7]:
class BuildDataset:
    def __init__(self, dataset: str, is_test: bool = True):
        self.ids = []
        self.is_test = is_test

        self.xmin, self.xmax = 0, 0

        self.data_tensor = self.load_volume(dataset)
        self.shape_orig = self.data_tensor.shape
        
    def normilize(self, image: np.ndarray) -> np.ndarray:
        if image.dtype != np.half:
            image = image.astype(np.half, copy=False)
            
        image -= self.xmin
        image /= (self.xmax - self.xmin)
        
        np.clip(image, 0, 1, out=image)
        return image
    
    @staticmethod
    def norm_by_percentile(
        volume: np.ndarray, low: float = 10, high: float = 99.8
    ) -> Tuple:
        xmin = np.percentile(volume, low)
        xmax = np.max([np.percentile(volume, high), 1])
        return int(xmin), int(xmax)

    def load_volume(self, dataset: str) -> np.ndarray:
        path = os.path.join(dataset, "images", "*.tif")
        
        dataset = sorted(glob(path)) if self.is_test else sorted(glob(path))[:192]

        for p_img in tqdm(dataset):
            path_ = p_img.split(os.path.sep)
            slice_id, _ = os.path.splitext(path_[-1])
            self.ids.append(f"{path_[-3]}_{slice_id}")

        volume = None

        for z, path in enumerate(tqdm(dataset)):
            image = cv2.imread(path, cv2.IMREAD_ANYDEPTH).astype(np.half, copy=False)
            
            if volume is None:
                volume = np.zeros((len(dataset), *image.shape[-2:]), dtype=np.float16)
            volume[z, :, :] = image
            
        self.xmin, self.xmax = self.norm_by_percentile(volume)
        return volume

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#Model architecture
class Conv2dBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(Conv2dBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding='same')
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding='same')
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        return x

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pool_type='avg', normalization=True):
        super(EncoderBlock, self).__init__()
        self.conv_block = Conv2dBlock(in_channels, out_channels)
        self.normalisation = normalization
        if pool_type == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pool_type == 'avg':
            self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x_conv = self.conv_block(x)
        x = self.pool(x_conv)
        
        if self.normalisation:
            x = self.batch_norm(x)
            
        x = self.dropout(x)
        return x, x_conv

class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Bottleneck, self).__init__()
        self.conv_block = Conv2dBlock(in_channels, out_channels)

    def forward(self, x):
        x = self.conv_block(x)
        return x

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, normalization=True):
        super(DecoderBlock, self).__init__()
        self.conv_transpose = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=2, padding=1, output_padding=1)
        self.conv_block = Conv2dBlock(in_channels, out_channels, kernel_size)
        self.normalisation = normalization
        
        self.batch_norm = nn.BatchNorm2d(in_channels)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, skip_connection):
        
        # Apply padding
        x = self.conv_transpose(x)
        x = torch.cat([x, skip_connection], dim=1)
        if self.normalisation:
            x = self.batch_norm(x)
            
        x = self.dropout(x)
        x = self.conv_block(x)
        return x
    

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.encoder1 = EncoderBlock(in_channels, 64)
        self.encoder2 = EncoderBlock(64, 128)
        self.encoder3 = EncoderBlock(128, 256, normalization=False)
        self.encoder4 = EncoderBlock(256, 512, normalization=False)
        self.bottleneck = Bottleneck(512, 1024)
        self.decoder1 = DecoderBlock(1024, 512, normalization=False)
        self.decoder2 = DecoderBlock(512, 256, normalization=False)
        self.decoder3 = DecoderBlock(256, 128)
        self.decoder4 = DecoderBlock(128, 64)
        self.output = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        enc1, conv1 = self.encoder1(x)
        enc2, conv2 = self.encoder2(enc1)
        enc3, conv3 = self.encoder3(enc2)
        enc4, conv4 = self.encoder4(enc3)
        bottleneck = self.bottleneck(enc4)
        dec1 = self.decoder1(bottleneck, conv4)
        dec2 = self.decoder2(dec1, conv3)
        dec3 = self.decoder3(dec2, conv2)
        dec4 = self.decoder4(dec3, conv1)
        output = self.output(dec4)
#         output = nn.functional.sigmoid(output)
        return output



def rle_encode(mask):
    """
    rle encoder (thanks to the community)
    """
    pixel = mask.flatten()
    pixel[pixel<0.5] = 0
    pixel[pixel>=0.5] = 1 
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 1'
    return rle

In [9]:
import re
# def rename_keys(state_dict, prefix):
#     return {k[len(prefix):]: v for k, v in state_dict.items() if k.startswith(prefix)}

def rename_keys(original_dict, pattern):
    new_dict = {}
    
    for old_key, value in original_dict.items():
        new_key = re.sub(pattern, '', old_key)
        
        new_dict[new_key] = value
    
    return new_dict

# Load state dictionary from checkpoint
updated_state_dict = rename_keys(torch.load(WEIGHTS_PATH, map_location="cpu")["state_dict"], "net.")

model = UNet(INPUT_CHANNELS, OUTPUT_CHANNELS)
model.load_state_dict(updated_state_dict)
model.to(device)

UNet(
  (encoder1): EncoderBlock(
    (conv_block): Conv2dBlock(
      (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (relu): ReLU()
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (encoder2): EncoderBlock(
    (conv_block): Conv2dBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (relu): ReLU()
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (batch_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (encoder3): EncoderBlock(
    (conv_block): Conv2dBlock(
      (conv1): Conv2d(128, 256, ke

In [10]:
rles, ids = [], []
datasets = sorted(glob(f"{MAIN_PATH}/test/*"))
inferer = SlidingWindowInferer(mode='gaussian', roi_size=(512,512))

for dataset in datasets:
    test_dataset = BuildDataset(dataset, is_test=True)

    input_tensor = torch.from_numpy(test_dataset.normilize(test_dataset.data_tensor)).unsqueeze(1)

    model.eval()
    with torch.no_grad():
        ids += test_dataset.ids
        for tensor in input_tensor:
            tensor = tensor.float().unsqueeze(0).to(device)
            output = inferer(inputs=tensor, network=model)

            rle_output = rle_encode(output.cpu())
            rles.append(rle_output)

100%|██████████| 3/3 [00:00<00:00, 21.63it/s]


In [11]:
submission = pd.DataFrame(rles, columns=['rle'], index=ids)

# Save DataFrame to a CSV file
submission.to_csv('submission.csv', index_label='id', columns=['rle'])

In [12]:
submission

,rle
kidney_5_0000,1 1
kidney_5_0001,1 1
kidney_5_0002,1 1
kidney_6_0000,1 1
kidney_6_0001,1 1
kidney_6_0002,1 1
